In [ ]:
import random
random.seed(42)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torchvision.utils import make_grid
from torchvision import transforms as T
from torchvision import models, datasets
from random import randint
from torchinfo import summary

from tqdm import tqdm
import os

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

In [ ]:

train_data = datasets.CIFAR100('./', train=True, download=True)

# Stick all the images together to form a 1600000 X 32 X 3 array
x = np.concatenate([np.asarray(train_data[i][0]) for i in range(len(train_data))])

# calculate the mean and std along the (0, 1) axes
mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0, 1))/255
# the the mean and std
mean=mean.tolist()
std=std.tolist()

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def show_batch(dataloader):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)    
    imshow(make_grid(images)) # Using Torchvision.utils make_grid function
    
def show_image(dataloader):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)
    random_num = randint(0, len(images)-1)
    imshow(images[random_num])
    print(images[random_num])
    label = labels[random_num]
    print(f'Label: {label}, Shape: {images[random_num].shape}')

In [ ]:
# Transformation - optional depending on future resnet implementation

# Define transformation sequence for image pre-processing
# If not using pre-trained model, normalize with 0.5, 0.5, 0.5 (mean and SD)
# If using pre-trained ImageNet, normalize with mean=[0.485, 0.456, 0.406], 
# std=[0.229, 0.224, 0.225])

train_transform = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ToTensor(),
    T.Normalize(mean, std, inplace=True)
])

test_transform = T.Compose([
                # T.Resize(256), # Resize images to 256 x 256
                # T.CenterCrop(224), # Center crop image
                # T.RandomHorizontalFlip(),
                T.ToTensor(),  # Converting cropped images to tensors
                T.Normalize(mean, std)
])


In [ ]:
batch_size = 128

In [ ]:
trainset = datasets.CIFAR100("./",
                                         train=True,
                                         download=True,
                                         transform=train_transform)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size, shuffle=True, num_workers=2,pin_memory=True)

testset = datasets.CIFAR100("./",
                                        train=False,
                                        download=True,
                                        transform=test_transform)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size*2,pin_memory=True, num_workers=2)

In [ ]:
show_image(train_loader)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

    
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
            nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded.view(-1, 48*4*4)

# class ResizeConv2d(nn.Module):

#     def __init__(self, in_channels, out_channels, kernel_size, scale_factor, mode='nearest'):
#         super().__init__()
#         self.scale_factor = scale_factor
#         self.mode = mode
#         self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=1)

#     def forward(self, x):
#         x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)
#         x = self.conv(x)
#         return x

# class BasicBlockEnc(nn.Module):

#     def __init__(self, in_planes, stride=1):
#         super().__init__()

#         planes = in_planes*stride

#         self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(planes)

#         if stride == 1:
#             self.shortcut = nn.Sequential()
#         else:
#             self.shortcut = nn.Sequential(
#                 nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(planes)
#             )

#     def forward(self, x):
#         out = torch.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         out += self.shortcut(x)
#         out = torch.relu(out)
#         return out

# class BasicBlockDec(nn.Module):

#     def __init__(self, in_planes, stride=1):
#         super().__init__()

#         planes = int(in_planes/stride)

#         self.conv2 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(in_planes)
#         # self.bn1 could have been placed here, but that messes up the order of the layers when printing the class

#         if stride == 1:
#             self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
#             self.bn1 = nn.BatchNorm2d(planes)
#             self.shortcut = nn.Sequential()
#         else:
#             self.conv1 = ResizeConv2d(in_planes, planes, kernel_size=3, scale_factor=stride)
#             self.bn1 = nn.BatchNorm2d(planes)
#             self.shortcut = nn.Sequential(
#                 ResizeConv2d(in_planes, planes, kernel_size=3, scale_factor=stride),
#                 nn.BatchNorm2d(planes)
#             )

#     def forward(self, x):
#         out = torch.relu(self.bn2(self.conv2(x)))
#         out = self.bn1(self.conv1(out))
#         out += self.shortcut(x)
#         out = torch.relu(out)
#         return out

# class ResNet9Enc(nn.Module):

#     def __init__(self, num_Blocks=[1,1,1,1], z_dim=10, nc=3):
#         super().__init__()
#         self.in_planes = 12
#         self.z_dim = z_dim
#         self.conv1 = nn.Conv2d(nc, 12, kernel_size=3, stride=2, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(12)
#         self.layer1 = self._make_layer(BasicBlockEnc, 24, num_Blocks[0], stride=2)
#         self.layer2 = self._make_layer(BasicBlockEnc, 48, num_Blocks[1], stride=2)

#     def _make_layer(self, BasicBlockEnc, planes, num_Blocks, stride):
#         strides = [stride] + [1]*(num_Blocks-1)
#         layers = []
#         for stride in strides:
#             layers += [BasicBlockEnc(self.in_planes, stride)]
#             self.in_planes = planes
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = torch.relu(self.bn1(self.conv1(x)))
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = x.view(-1, 48*4*4)
#         x = x.view(x.size(0), -1)

#         return x

# class ResNet9Dec(nn.Module):

#     def __init__(self, num_Blocks=[1,1,1,1], z_dim=10, nc=3):
#         super().__init__()
#         self.in_planes = 48

#         self.layer2 = self._make_layer(BasicBlockDec, 24, num_Blocks[1], stride=2)
#         self.layer1 = self._make_layer(BasicBlockDec, 12, num_Blocks[0], stride=2)
#         self.conv1 = ResizeConv2d(12, nc, kernel_size=3, scale_factor=1)

#     def _make_layer(self, BasicBlockDec, planes, num_Blocks, stride):
#         strides = [stride] + [1]*(num_Blocks-1)
#         layers = []
#         for stride in reversed(strides):
#             layers += [BasicBlockDec(self.in_planes, stride)]
#         self.in_planes = planes
#         return nn.Sequential(*layers)

#     def forward(self, z):
#         x = z.view(z.size(0), 768, 1, 1)
#         x = F.interpolate(x, scale_factor=2)
#         x = x.reshape((z.size(0), 48, 8, 8))
#         x = self.layer2(x)
#         x = self.layer1(x)
#         x = torch.sigmoid(self.conv1(x))
#         x = x.view(x.size(0), 3, 32, 32)
#         return x

# class Autoencoder(nn.Module):

#     def __init__(self, z_dim):
#         super().__init__()
#         self.encoder = ResNet9Enc(z_dim=z_dim)
#         self.decoder = ResNet9Dec(z_dim=z_dim)

#     def forward(self, x):
#         z = self.encoder(x)
#         x = self.decoder(z)
#         return x, z

def test():
    net = Autoencoder()
    net = net.to(device)
    item = iter(train_loader)
    img, _ = next(item)

    img = img[0]
    img = img[None, :]
    img = img.to(device)
    img_cons, rep  = net(img)
    std_t = torch.tensor(std).to(device)
    mean_t = torch.tensor(mean).to(device)
    img_cons = img_cons * std_t[:, None, None] + mean_t[:, None, None]
    img = img * std_t[:, None, None] + mean_t[:, None, None]
    # print(rep)
    print(rep.shape, "  ", img_cons.shape)
    temp_r = imshow(img.cpu()[0])
    imshow(img_cons.detach().cpu()[0])

In [ ]:
test()

In [ ]:
dataloaders = {
    'train': train_loader,
    'val': test_loader,
    'test': test_loader
}
dataset_sizes = {
    'train': len(train_loader.dataset),
    'val': len(test_loader.dataset),
    'test': len(test_loader.dataset),
}

In [ ]:
import torch.nn.functional as F
def sparse_loss(autoencoder, images):
    loss = 0
    values = images
    model_children = list(autoencoder.children())
    for i in range(len(model_children)):
        values = F.relu((model_children[i](values)))
        loss += torch.mean(torch.abs(values))
    return loss

In [ ]:
import time
import copy
from torch import autograd

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, _ in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    reconstructed_img, hidden_rep = model(inputs)
                    
                    # l1_loss = sparse_loss(model, inputs) * 0.001
                    loss = criterion(reconstructed_img, inputs)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        with autograd.detect_anomaly():
                            loss.backward()
                            optimizer.step()
#                         for name, param in model_ft.named_parameters():
#                             print(name, param.grad.norm())

                # statistics
                running_loss += loss.item() * inputs.size(0)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss}')          

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
#             if phase == 'val':
                print('Saving..')
                state = {
                    'model': model.state_dict(),
                    'loss': epoch_loss,
                    'epoch': epoch,
                }
                # if not os.path.isdir('checkpoint'):
                #     os.mkdir('checkpoint')
                torch.save(state, './tiny_ae_layers5.pth')
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val loss: {best_loss:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
from torch.optim import lr_scheduler
model_ft = Autoencoder()
model_ft = model_ft.to(device)

criterion = nn.MSELoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_ft, milestones=[60, 120, 160], gamma=0.2)


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

In [ ]:
for name, param in model_ft.named_parameters():
    if param.grad != None:
        print(name, param.grad.norm())

In [ ]:
import cv2
device = 'cuda'
model_ft.eval()
item = iter(train_loader)
img, _ = next(item)

img = img[0]
img = img[None, :]
img = img.to(device)
img_cons, rep  = model_ft(img)
std_t = torch.tensor(std).to(device)
mean_t = torch.tensor(mean).to(device)
img_cons = img_cons * std_t[:, None, None] + mean_t[:, None, None]
img = img * std_t[:, None, None] + mean_t[:, None, None]
print(rep.shape, "  ", img_cons.shape)
temp_r = imshow(img.cpu()[0])
imshow(img_cons.detach().cpu()[0])
print(rep)

In [ ]:
train_image_embeddings = []
train_image_labels = []

def populate_embedding_lable_list(model):
    global train_image_embeddings, train_image_labels

    model.eval()   # Set model to evaluate mode
    # Iterate over data.
    for inputs, labels in tqdm(dataloaders['train']):
        inputs = inputs.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            reconstructed_img, hidden_rep = model(inputs)
            train_image_embeddings.extend(hidden_rep)
            train_image_labels.extend(labels)

In [ ]:
populate_embedding_lable_list(model_ft)

In [ ]:
train_image_labels = torch.flatten(torch.tensor(train_image_labels))
# torch.save(train_image_labels, '/content/drive/MyDrive/TDML_Project/train_image_labels.pt')

train_image_embeddings_tensor = torch.stack(train_image_embeddings)
# torch.save(train_image_embeddings_tensor, '/content/drive/MyDrive/TDML_Project/train_image_embeddings_tensor.pt')

In [ ]:
print(len(sorted(set(list(train_loader.dataset.targets)))))
print(len(sorted(set(list(train_image_labels.numpy())))))
print(train_image_labels.shape, " ", train_image_embeddings_tensor.shape)

In [ ]:
from collections import Counter

def find_indices(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]
  
image_label_list = list(train_image_labels.numpy())
classes_to_index = {i: [] for i in range(100)}
for key in classes_to_index.keys():
    indexes = find_indices(image_label_list, key)
    random_40 = random.sample(indexes,40)
    classes_to_index[key] = random_40

print(classes_to_index)

In [ ]:
import itertools
final_indices = list(classes_to_index.values())
final_indices = sorted([item for sublist in final_indices for item in sublist])
print(final_indices)
print(len(final_indices))

In [ ]:
similarity_matrix = np.zeros((100,100,2))
# device = 'cpu'
# print(similarity_matrix[0])
# print(similarity_matrix)

In [ ]:
cosine_sim = nn.CosineSimilarity(dim=0, eps=1e-8)
print(train_image_embeddings[0].shape)
cosine_sim(train_image_embeddings[2957], train_image_embeddings[20364].T)

In [ ]:
for act_i in tqdm(range(len(final_indices))):
    for act_j in range(act_i+1, len(final_indices)):
        i, j = final_indices[act_i], final_indices[act_j]
        class_i, class_j = train_image_labels[i], train_image_labels[j]
        emb_i, emb_j = train_image_embeddings[i], train_image_embeddings[j]
        sim_score = cosine_sim(emb_i, emb_j).detach().item()
    
        if class_i != class_j:
            similarity_matrix[class_i][class_j][0] += sim_score
            similarity_matrix[class_i][class_j][1] += 1

            similarity_matrix[class_j][class_i][0] += sim_score
            similarity_matrix[class_j][class_i][1] += 1
        else:
            similarity_matrix[class_i][class_i][0] += sim_score
            similarity_matrix[class_i][class_i][1] += 1

In [ ]:
avg_sim = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        if i == j:
            avg_sim[i][j] = similarity_matrix[i][j][0]/similarity_matrix[i][j][1] + 0.0225
        else:
            avg_sim[i][j] = similarity_matrix[i][j][0]/similarity_matrix[i][j][1]


In [ ]:
import torch.nn.functional as F
x_distribution = F.softmax(torch.tensor(avg_sim), dim = 1)

In [ ]:
x_distribution[0]

In [ ]:
avg_sim[0]

In [ ]:
count = 0
for i in range(len(x_distribution)):
    if i == torch.argmax(x_distribution[i]):
        count += 1
        # print(i)
print(count)

In [ ]:
torch.save(avg_sim, './ae_similarity_mat.pt')